https://www.cerebrium.ai/blog/improve-stable-diffusion-inference-by-50-with-tensorrt-or-aitemplate

Установим необходимые для работы библиотеки

In [1]:
!pip install setuptools pip --user

!pip install nvidia-pyindex

!pip install nvidia-tensorrt

!pip install pycuda

!pip install transformers diffusers scipy accelerate

  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-pyindex: filename=nvidia_pyindex-1.0.9-py3-none-any.whl size=8418 sha256=f577fc74a121be2f6cbefa2aae73dc86807a545935dd74977a14cd0e32d21da0
  Stored in directory: /root/.cache/pip/wheels/2c/af/d0/7a12f82cab69f65d51107f48bcd6179e29b9a69a90546332b3
Successfully built nvidia-pyindex
  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-8.6.1.post1-py2.py3-none-any.whl size=17283 sha256=1617ed0f604f098bf9c428114ffcfd194e733cfe2b274e9ba7085a54adc96c92
  Stored in directory: /root/.cache/pip/wheels/f4/c8/0e/b79b08e45752491b9acfdbd69e8a609e8b2ed7640dda5a3e59
Successfully built tensorrt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

SD состоит из трех частей:  
 - Variational autoencoder   
 - UNet  
 - CLIP text encoder   
Поскольку 90% всего времени работы сети занимает UNet, то имеенеет смысл оптимизировать именно эту часть.

Скачаем и распакуем UNet модель с huggingface

In [2]:
!wget https://huggingface.co/kamalkraj/stable-diffusion-v1-4-onnx/resolve/main/models.tar.gz

!tar -xf models.tar.gz

--2023-12-16 13:31:46--  https://huggingface.co/kamalkraj/stable-diffusion-v1-4-onnx/resolve/main/models.tar.gz
Resolving huggingface.co (huggingface.co)... 18.239.50.80, 18.239.50.103, 18.239.50.16, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.80|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/32/db/32dbcbc7a65fabe86a8cc5a1e7e461df46c92556ebe53adcf62dc0521861db09/c0dffa0cc37e080a0bf5c1d9bdc62fe7895cd13edcd4efb43a3ee25f387b6955?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27models.tar.gz%3B+filename%3D%22models.tar.gz%22%3B&response-content-type=application%2Fgzip&Expires=1702992706&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMjk5MjcwNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8zMi9kYi8zMmRiY2JjN2E2NWZhYmU4NmE4Y2M1YTFlN2U0NjFkZjQ2YzkyNTU2ZWJlNTNhZGNmNjJkYzA1MjE4NjFkYjA5L2MwZGZmYTBjYzM3ZTA4MGEwYmY1YzFkOWJkYzYyZmU3O

Теперь перековрертируем скаченную модель в формат для TensorRt из ONNX.

Импортируем библиотеки

In [3]:
import torch
import tensorrt as trt
import os, sys, argparse
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from time import time

Путь к начальной модели

In [4]:
onnx_model = "./models/unet/1/unet.onnx"
engine_filename = "unet_new.engine"

Сначала мы создаем TensorRt engine из ONNX модели и используем некоторые оптимизации, вроде того, что имзенеям такие параметры, как: precision mode, maximum batch size, and maximum workspace size  
Затем мы сохраняем полученный TensorRt engine в файл.

In [5]:
def convert_model():
    batch_size = 1
    # параметры картинки
    height = 512
    width = 512
    latents_shape = (batch_size*2, 4, height // 8, width // 8)
    embed_shape = (batch_size*2,64,768)
    timestep_shape = (batch_size,)

    TRT_LOGGER = trt.Logger(trt.Logger.INFO) # для логгирования инициализируем Logger у TensorRt
    TRT_BUILDER = trt.Builder(TRT_LOGGER) # инициализируем билдер и передаем ему логгер
    # EXPLICIT_BATCH : Specify that the network should be created with an explicit batch dimension.
    TRT_NETWORK = TRT_BUILDER.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)) # создвем сеть

    onnx_parser = trt.OnnxParser(TRT_NETWORK, TRT_LOGGER) # инициализируем парсер onnx для чтения данных из файла
    parse_success = onnx_parser.parse_from_file(onnx_model)
    for idx in range(onnx_parser.num_errors): # проверяем на налицие ошибок при чтении
        print(onnx_parser.get_error(idx))
    if not parse_success:
        sys.exit('ONNX model parsing failed')
    print("Load Onnx model done")

    config = TRT_BUILDER.create_builder_config()
    profile = TRT_BUILDER.create_optimization_profile()
    profile.set_shape("sample", latents_shape, latents_shape, latents_shape)
    profile.set_shape("encoder_hidden_states", embed_shape, embed_shape, embed_shape)
    profile.set_shape("timestep", timestep_shape, timestep_shape, timestep_shape)
    config.add_optimization_profile(profile)

    config.set_flag(trt.BuilderFlag.FP16)
    serialized_engine = TRT_BUILDER.build_serialized_network(TRT_NETWORK, config)

    # сохраняем TrT модель в файл
    with open(engine_filename, 'wb') as f:
        f.write(serialized_engine)
    print(f'Engine is saved to {engine_filename}')

In [6]:
# Конвертируем модель
convert_model()

Load Onnx model done
Engine is saved to unet_new.engine


Теперь воспользуемся полученной моделью

Импортирем необходимые библиотеки

In [7]:
import torch
from tqdm import tqdm
from PIL import Image
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL
from diffusers import LMSDiscreteScheduler
from torch import autocast
import argparse
import time


Функция получения аргументов для сети

In [8]:
def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--prompt",
        default="Super Mario learning to fly in an airport, Painting by Leonardo Da Vinci",
        help="input prompt",
    )
    parser.add_argument(
        "--trt_unet_save_path",
        default="./unet.engine",
        type=str,
        help="TensorRT unet saved path",
    )
    parser.add_argument("--batch_size", default=1, type=int, help="batch size")
    parser.add_argument(
        "--img_size", default=(512, 512), help="Unet input image size (h,w)"
    )
    parser.add_argument(
        "--max_seq_length", default=64, help="Maximum sequence length of input text"
    )
    parser.add_argument(
        "--benchmark",
        action="store_true",
        help="Running benchmark by average num iteration",
    )
    parser.add_argument(
        "--n_iters", default=50, help="Running benchmark by average num iteration"
    )

    return parser.parse_args()

Построим класс для работы с моделью

In [ ]:
from trt_model import TRTModel
class TrtDiffusionModel:
    def __init__(self, args):
        self.device = torch.device("cuda") # устанавливаем режим работы на графическом процессоре
        self.unet = TRTModel(args.trt_unet_save_path) # указываем путь к сохраненной сети UNet
        # Устанавливаем остальные параметры SD - дефолтными и неоптимизированными
        # (См составные части SD)
        self.vae = AutoencoderKL.from_pretrained(
            "stabilityai/stable-diffusion-2-1", subfolder="vae", use_auth_token=True
        ).to(self.device)
        self.tokenizer = CLIPTokenizer.from_pretrained(
            "stabilityai/stable-diffusion-2-1", subfolder="tokenizer", use_auth_token=True
        )
        self.text_encoder = CLIPTextModel.from_pretrained(
            "openai/clip-vit-large-patch14"
        ).to(self.device)

        self.scheduler = LMSDiscreteScheduler(
            beta_start=0.00085,
            beta_end=0.012,
            beta_schedule="scaled_linear",
            num_train_timesteps=1000,
        )

    def predict(
        self, prompts, num_inference_steps=50, height=512, width=512, max_seq_length=64
    ):
        guidance_scale = 7.5
        batch_size = 1
        text_input = self.tokenizer(
            prompts,
            padding="max_length",
            max_length=max_seq_length,
            truncation=True,
            return_tensors="pt",
        )
        text_embeddings = self.text_encoder(text_input.input_ids.to(self.device))[0]
        uncond_input = self.tokenizer(
            [""] * batch_size,
            padding="max_length",
            max_length=max_seq_length,
            return_tensors="pt",
        )
        uncond_embeddings = self.text_encoder(uncond_input.input_ids.to(self.device))[0]
        text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

        latents = torch.randn((batch_size, 4, height // 8, width // 8)).to(self.device)
        self.scheduler.set_timesteps(num_inference_steps)

        latents = latents * self.scheduler.sigmas[0]
        with torch.inference_mode(), autocast("cuda"):
            for i, t in tqdm(enumerate(self.scheduler.timesteps)):
                latent_model_input = torch.cat([latents] * 2)
                sigma = self.scheduler.sigmas[i]
                latent_model_input = latent_model_input / ((sigma**2 + 1) ** 0.5)

                # predict the noise residual
                inputs = [
                    latent_model_input,
                    torch.tensor([t]).to(self.device),
                    text_embeddings,
                ]
                noise_pred, duration = self.unet(inputs, timing=True)
                noise_pred = torch.reshape(noise_pred[0], (batch_size * 2, 4, 64, 64))

                # perform guidance
                noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
                noise_pred = noise_pred_uncond + guidance_scale * (
                    noise_pred_text - noise_pred_uncond
                )

                # compute the previous noisy sample x_t -> x_t-1
                latents = self.scheduler.step(noise_pred.cuda(), t, latents)[
                    "prev_sample"
                ]

            # scale and decode the image latents with vae
            latents = 1 / 0.18215 * latents
            image = self.vae.decode(latents).sample
        return image

Начинаем работать с самой моделью

Получаем аргументы и инициализируем с помощью их модель

In [ ]:
args = get_args()
model = TrtDiffusionModel(args)

Проверяем установлен ли флаг на бенчмарк и проводим проверку, если таковой установлен на True

In [ ]:
if args.benchmark:
    n_iters = args.n_iters
    # warm up
    for i in range(3):
        image = model.predict(
            prompts=args.prompt,
            num_inference_steps=50,
            height=args.img_size[0],
            width=args.img_size[1],
            max_seq_length=args.max_seq_length,
        )
else:
    n_iters = 1

Запускаем predict(...) модели с заданными параметрами

In [ ]:
start = time.time()
for i in tqdm(range(n_iters)):
    image = model.predict(
        prompts=args.prompt,
        num_inference_steps=50,
        height=args.img_size[0],
        width=args.img_size[1],
        max_seq_length=args.max_seq_length,
    )
end = time.time()

Выводим получившееся изображение и результаты по времени

In [ ]:
if args.benchmark:
    print("Average inference time is: ", (end - start) / n_iters)
image = (image / 2 + 0.5).clamp(0, 1)
image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
images = (image * 255).round().astype("uint8")
pil_images = [Image.fromarray(image) for image in images]
pil_images[0].save("image_generated.png")